In [1]:
import os, sys
sys.path.append("../../common/")
from python_tools import *

python tools loaded.


In [2]:
%matplotlib notebook

In [3]:
TRUTH_DF_FILE = "/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/mcana_dfs.h5"
PELEE_1ENP_FILES = ["/Users/wketchum/Data/MicroBooNE/FakeData2020/PeLEE/fakedata1/data_1eNp_fakedata1.root"]
PELEE_1E0P_FILES = ["/Users/wketchum/Data/MicroBooNE/FakeData2020/PeLEE/1e0p/prod_uboone_nu2020_fakedata_set1_run1_reco2_v08_00_00_41_reco2_fakedata1_1e0p_BDT_sbnfit.root",
                    "/Users/wketchum/Data/MicroBooNE/FakeData2020/PeLEE/1e0p/prod_uboone_nu2020_fakedata_set1_run3b_reco2_v08_00_00_41_reco2_fakedata1_1e0p_BDT_sbnfit.root"]
DLLEE_FILES = ["/Users/wketchum/Data/MicroBooNE/FakeData2020/DLeLEE/sel1e1p_fakedata_set1_run1_eventlist.txt",
               "/Users/wketchum/Data/MicroBooNE/FakeData2020/DLeLEE/sel1e1p_fakedata_set1_run3_eventlist.txt"]
GLEE_1G1P_FILES = ["/Users/wketchum/Data/MicroBooNE/FakeData2020/gLEE/sbnfit_1g1pMar2020_v4_stage_6_FakeDataRedo1.root"]
GLEE_1G0P_FILES = ["/Users/wketchum/Data/MicroBooNE/FakeData2020/gLEE/sbnfit_1g0pMar2020_stage_4_FakeDataRedo1.root"]

In [4]:
df_n = pd.read_hdf(TRUTH_DF_FILE,"ev_df")
df_n = df_n.reset_index()
df_n = df_n.astype({"run":"int64","subrun":"int64","event":"int64"})
print(len(df_n))

466225


In [5]:
pelee_1eNp_df = pd.concat([uproot.open(f)["NeutrinoSelectionFilter"].pandas.df() for f in PELEE_1ENP_FILES])
pelee_1e0p_df = pd.concat([uproot.open(f)["NeutrinoSelectionFilter"].pandas.df() for f in PELEE_1E0P_FILES])
pelee_1eNp_df = pelee_1eNp_df.rename(columns={"sub":"subrun","evt":"event"})
pelee_1e0p_df = pelee_1e0p_df.rename(columns={"sub":"subrun","evt":"event"})

In [6]:
dl_df = pd.concat([pd.read_csv(f,names=["run","subrun","event","reco_nu_e"],header=None) for f in DLLEE_FILES])

In [7]:
glee_1g1p_df = pd.concat([uproot.open(f)["singlephoton/vertex_tree"].pandas.df(flatten=False) for f in GLEE_1G1P_FILES])
glee_1g0p_df = pd.concat([uproot.open(f)["singlephoton/vertex_tree"].pandas.df(flatten=False) for f in GLEE_1G0P_FILES])
glee_1g1p_df = glee_1g1p_df.rename(columns={"run_number":"run","subrun_number":"subrun","event_number":"event"})
glee_1g0p_df = glee_1g0p_df.rename(columns={"run_number":"run","subrun_number":"subrun","event_number":"event"})

In [19]:
def find_truth(df_t,r,sr,ev):
    return df_t.query("run==%d and subrun==%d and event==%d"%(r,sr,ev))[["run","subrun","event",
                                                                         "n_mu","n_e","n_p_40MeV",
                                                                         "n_pi0","n_gamma","e","mode",
                                                                         "start_x","start_y","start_z"]]

In [17]:
def print_breakdown(df_n,selection_df,selection_label="Selection",verbose=True):
    n_1eNp = 0
    n_1e0p = 0
    n_1e1p = 0
    n_1gNp = 0
    n_1g1p = 0
    n_1g0p = 0
    n_ncpi0 = 0
    n_ccpi0 = 0
    n_cc_other = 0
    n_nc_other = 0
    n_notfound = 0
    for i,ev in selection_df.iterrows():
        mct = find_truth(df_n,ev["run"],ev["subrun"],ev["event"])
        if(len(mct)==0):
            print("Error: cannot find run %d, subrun %d, event %d"%(ev["run"],ev["subrun"],ev["event"]))
            n_notfound+=1
            if(verbose):
                print("Run %d, Subrun %d, Event %d: Not found."%(ev["run"],ev["subrun"],ev["event"]))
            continue;
        
        mct = mct.iloc[0]

        if(mct["n_e"]>0):
            if(mct["n_p_40MeV"]>0): 
                n_1eNp+=1
                if(mct["n_p_40MeV"]==1): 
                    n_1e1p+=1
                    if(verbose):
                        print("Run %d, Subrun %d, Event %d: 1e1p. E_nu=%f, vtx_nu=(%f,%f,%f)"%
                              (ev["run"],ev["subrun"],ev["event"],
                               mct["e"],mct["start_x"],mct["start_y"],mct["start_z"]))
                else:
                    if(verbose):
                        print("Run %d, Subrun %d, Event %d: 1e,>1p (1eNp). E_nu=%f, vtx_nu=(%f,%f,%f)"%
                              (ev["run"],ev["subrun"],ev["event"],
                               mct["e"],mct["start_x"],mct["start_y"],mct["start_z"]))
                        
            elif(mct["n_p_40MeV"]==0): 
                n_1e0p+=1
                if(verbose):
                    print("Run %d, Subrun %d, Event %d: 1e0p. E_nu=%f, vtx_nu=(%f,%f,%f)"%
                          (ev["run"],ev["subrun"],ev["event"],
                          mct["e"],mct["start_x"],mct["start_y"],mct["start_z"]))

        elif(mct["n_gamma"]>0):
            if(mct["n_p_40MeV"]>0): 
                n_1gNp+=1
                if(mct["n_p_40MeV"]==1):
                    n_1g1p+=1
                    if(verbose):
                        print("Run %d, Subrun %d, Event %d: 1g1p. E_nu=%f, vtx_nu=(%f,%f,%f)"%
                              (ev["run"],ev["subrun"],ev["event"],
                              mct["e"],mct["start_x"],mct["start_y"],mct["start_z"]))                
                    else:
                        if(verbose):
                            print("Run %d, Subrun %d, Event %d: 1g,>1p (1gNp). E_nu=%f, vtx_nu=(%f,%f,%f)"%
                                  (ev["run"],ev["subrun"],ev["event"],
                                  mct["e"],mct["start_x"],mct["start_y"],mct["start_z"])) 
                                  
            elif(mct["n_p_40MeV"]==0): 
                n_1g0p+=1
                if(verbose):
                    print("Run %d, Subrun %d, Event %d: 1g0p. E_nu=%f, vtx_nu=(%f,%f,%f)"%
                          (ev["run"],ev["subrun"],ev["event"],
                          mct["e"],mct["start_x"],mct["start_y"],mct["start_z"])) 
        elif(mct["n_pi0"]>0):
            if(mct["n_mu"]==0):
                n_ncpi0+=1
                if(verbose):
                    print("Run %d, Subrun %d, Event %d: NC pi0. E_nu=%f, vtx_nu=(%f,%f,%f)"%
                          (ev["run"],ev["subrun"],ev["event"],
                          mct["e"],mct["start_x"],mct["start_y"],mct["start_z"])) 
            if(mct["n_mu"]==1):
                n_ccpi0+=1
                if(verbose):
                    print("Run %d, Subrun %d, Event %d: CC pi0. E_nu=%f, vtx_nu=(%f,%f,%f)"%
                          (ev["run"],ev["subrun"],ev["event"],
                          mct["e"],mct["start_x"],mct["start_y"],mct["start_z"])) 
        elif(mct["n_mu"]>0):
            n_cc_other+=1
            if(verbose):
                print("Run %d, Subrun %d, Event %d: CC other. E_nu=%f, vtx_nu=(%f,%f,%f), mode=%d"%
                      (ev["run"],ev["subrun"],ev["event"],
                      mct["e"],mct["start_x"],mct["start_y"],mct["start_z"],
                      mct["mode"]))
        else:
            n_nc_other+=1
            if(verbose):
                print("Run %d, Subrun %d, Event %d: NC other. E_nu=%f, vtx_nu=(%f,%f,%f), mode=%d"%
                      (ev["run"],ev["subrun"],ev["event"],
                      mct["e"],mct["start_x"],mct["start_y"],mct["start_z"],
                      mct["mode"]))
    
    print("%s Total Events: %d"%(selection_label,len(selection_df)))
    print("\t1e0p: %d, 1e1p: %d, 1eNp (incl. 1e1p): %d"%(n_1e0p,n_1e1p,n_1eNp))
    print("\t1g0p: %d, 1g1p: %d, 1gNp (incl. 1g1p): %d"%(n_1g0p,n_1g1p,n_1gNp))
    print("\tNC pi0: %d, CC pi0: %d"%(n_ncpi0,n_ccpi0))
    print("\tOther CC: %d"%n_cc_other)
    print("\tOther NC: %d"%n_cc_other)


In [12]:
print_breakdown(df_n,pelee_1eNp_df,"PeLEE 1eNP Selection")

Run 6693842, Subrun 4530, Event 3: 1e1p. E_nu=0.631774, vtx_nu=(15.467068,-54.280884,342.506317)
Run 6693842, Subrun 7304, Event 24: 1e1p. E_nu=0.405610, vtx_nu=(186.864731,-35.253265,356.324371)
Run 6693859, Subrun 12, Event 2: 1e1p. E_nu=1.718942, vtx_nu=(176.739883,45.656372,596.481567)
Run 6693842, Subrun 410, Event 1: 1e1p. E_nu=2.233982, vtx_nu=(25.485619,59.954109,524.238770)
Run 6693859, Subrun 1369, Event 12: 1e1p. E_nu=0.856868, vtx_nu=(70.929810,-40.477257,172.639450)
Run 6693842, Subrun 2449, Event 5: CC other. E_nu=1.401885, vtx_nu=(38.111370,24.147839,798.568298)
Run 6693859, Subrun 2009, Event 16: 1e1p. E_nu=0.683497, vtx_nu=(122.302109,-61.251755,531.372253)
Run 6693859, Subrun 1053, Event 10: NC pi0. E_nu=0.970553, vtx_nu=(115.109467,62.598160,454.395538)
Run 6693842, Subrun 5586, Event 15: CC pi0. E_nu=1.085787, vtx_nu=(49.808006,-29.157391,783.547913)
Run 6693859, Subrun 1812, Event 5: 1e,>1p (1eNp). E_nu=1.585040, vtx_nu=(108.704010,-67.987946,398.419891)
Run 669385

In [13]:
print_breakdown(df_n,pelee_1e0p_df,"PeLEE 1e0P Selection")

Run 6693842, Subrun 7304, Event 27: 1e1p. E_nu=0.396108, vtx_nu=(85.439018,94.359428,140.611786)
Run 6693842, Subrun 5208, Event 5: 1e0p. E_nu=0.674346, vtx_nu=(101.372025,-49.697636,886.893921)
Run 6693842, Subrun 5519, Event 1: NC pi0. E_nu=0.929973, vtx_nu=(160.364517,-64.850784,234.319351)
Run 6693842, Subrun 2498, Event 2: 1e0p. E_nu=1.311130, vtx_nu=(207.654190,-44.269680,244.292252)
Run 6693859, Subrun 1649, Event 6: CC pi0. E_nu=1.471978, vtx_nu=(-47.888969,7.567313,424.658051)
Run 6693842, Subrun 8711, Event 3: NC pi0. E_nu=1.482962, vtx_nu=(66.182877,-101.714043,160.232559)
Run 6693842, Subrun 4413, Event 16: CC other. E_nu=0.970033, vtx_nu=(149.889801,70.565147,151.612457)
Run 6693842, Subrun 7784, Event 1: 1e0p. E_nu=0.737455, vtx_nu=(44.492027,-21.926159,255.422363)
Run 6693842, Subrun 8283, Event 5: 1e0p. E_nu=0.318139, vtx_nu=(47.284664,-85.753181,971.502258)
Run 6693842, Subrun 5542, Event 1: 1e1p. E_nu=1.345412, vtx_nu=(104.011864,-23.895922,96.095856)
Run 6693842, Sub

In [20]:
print_breakdown(df_n,dl_df,"DL LEE 1e1P Selection",True)

Run 6693842, Subrun 3230, Event 10: 1e1p. E_nu=0.328623, vtx_nu=(226.934021,-87.072906,258.978210)
Run 6693842, Subrun 3230, Event 16: 1e1p. E_nu=0.323824, vtx_nu=(124.378990,-20.707279,439.826874)
Run 6693859, Subrun 567, Event 10: 1e1p. E_nu=0.277444, vtx_nu=(229.262146,21.034636,351.297943)
Run 6693859, Subrun 1053, Event 10: NC pi0. E_nu=0.970553, vtx_nu=(115.109467,62.598160,454.395538)
Run 6693859, Subrun 1153, Event 13: CC other. E_nu=1.450441, vtx_nu=(4.750628,-102.172966,-42.839897), mode=0
Run 6693859, Subrun 868, Event 14: 1e,>1p (1eNp). E_nu=0.856013, vtx_nu=(132.978836,75.618423,568.652100)
Run 6693859, Subrun 265, Event 2: 1e1p. E_nu=1.089687, vtx_nu=(208.309494,-96.471474,200.296906)
Run 6693842, Subrun 40, Event 17: 1e,>1p (1eNp). E_nu=0.205381, vtx_nu=(163.952591,98.121346,394.483429)
Run 6693842, Subrun 9301, Event 2: 1e1p. E_nu=0.354521, vtx_nu=(105.642654,28.699385,871.294128)
Run 6693842, Subrun 6172, Event 14: 1g1p. E_nu=0.642499, vtx_nu=(217.528427,50.168800,102.

In [15]:
print_breakdown(df_n,glee_1g1p_df,"gLEE 1g1P Selection")

Run 6693842, Subrun 5097, Event 17: 1g1p. E_nu=0.939435, vtx_nu=(54.106716,-61.964970,905.029297)
Run 7165574, Subrun 7688, Event 7: NC pi0. E_nu=1.096632, vtx_nu=(132.453369,62.574577,815.768677)
Run 6693842, Subrun 3291, Event 10: 1g1p. E_nu=1.061345, vtx_nu=(39.425957,14.357633,281.620087)
Run 6693842, Subrun 3497, Event 1: 1g1p. E_nu=1.138696, vtx_nu=(101.894104,60.423069,865.805908)
Run 7165574, Subrun 7755, Event 4: 1g1p. E_nu=0.844507, vtx_nu=(62.110340,12.630075,277.188843)
Run 6693842, Subrun 6737, Event 3: 1g1p. E_nu=1.181573, vtx_nu=(18.989454,40.715969,415.550476)
Run 6693842, Subrun 7360, Event 23: 1g1p. E_nu=0.609091, vtx_nu=(229.125366,-4.969529,565.453735)
Run 7165574, Subrun 6410, Event 1: 1g1p. E_nu=1.402948, vtx_nu=(1.942278,-16.728468,423.630157)
Run 6693859, Subrun 1837, Event 18: 1g1p. E_nu=1.257980, vtx_nu=(48.621178,-31.736862,512.474487)
Run 6693842, Subrun 7891, Event 1: 1g1p. E_nu=0.759902, vtx_nu=(200.616318,-78.007622,594.682434)
Run 6693842, Subrun 9312, E

In [16]:
print_breakdown(df_n,glee_1g0p_df,"gLEE 1g0P Selection")

Run 7165574, Subrun 8956, Event 12: 1g0p. E_nu=1.180011, vtx_nu=(253.180481,73.403419,545.782043)
Run 7165574, Subrun 8036, Event 4: CC pi0. E_nu=0.874209, vtx_nu=(-2.691910,1.272083,652.343872)
Run 6693842, Subrun 1868, Event 2: CC pi0. E_nu=1.112750, vtx_nu=(115.766068,66.055878,477.773834)
Run 6693859, Subrun 883, Event 16: 1e1p. E_nu=1.082596, vtx_nu=(179.970993,-89.135963,721.492188)
Run 7165592, Subrun 1529, Event 6: NC pi0. E_nu=1.561587, vtx_nu=(112.966599,-14.442729,853.011047)
Run 7165574, Subrun 4413, Event 15: NC pi0. E_nu=0.915085, vtx_nu=(168.791916,16.868551,212.955750)
Run 6693842, Subrun 5607, Event 16: CC pi0. E_nu=4.304123, vtx_nu=(133.671585,-111.493874,92.180313)
Run 6693859, Subrun 1759, Event 9: NC pi0. E_nu=1.258993, vtx_nu=(73.184830,-63.297607,600.987854)
Run 7165592, Subrun 2065, Event 9: NC pi0. E_nu=0.612741, vtx_nu=(77.994568,-54.339211,712.520813)
Run 6693859, Subrun 2043, Event 11: NC pi0. E_nu=0.869317, vtx_nu=(17.597292,36.989956,466.079926)
Run 669385

Run 6693842, Subrun 6742, Event 10: CC pi0. E_nu=1.166655, vtx_nu=(217.464661,-57.110611,567.942688)
Run 7165574, Subrun 7995, Event 11: CC pi0. E_nu=0.915012, vtx_nu=(92.865707,7.777111,88.306931)
Run 7165574, Subrun 9698, Event 18: CC pi0. E_nu=1.985047, vtx_nu=(262.654144,-66.917809,774.954834)
Run 7165574, Subrun 5184, Event 26: 1g0p. E_nu=1.771324, vtx_nu=(207.999435,8.401753,171.284012)
Run 6693842, Subrun 5620, Event 12: CC other. E_nu=1.014135, vtx_nu=(68.624260,66.021408,757.414246)
Run 6693859, Subrun 1624, Event 6: 1g1p. E_nu=1.275393, vtx_nu=(44.673229,-37.929867,638.315796)
Run 7165574, Subrun 8411, Event 14: CC pi0. E_nu=1.008576, vtx_nu=(131.528748,-17.448774,91.952728)
Run 7165574, Subrun 846, Event 21: CC pi0. E_nu=1.011362, vtx_nu=(60.536552,67.706070,701.673645)
Run 6693842, Subrun 4351, Event 4: NC pi0. E_nu=0.823815, vtx_nu=(143.997101,-85.533791,200.827087)
Run 7165574, Subrun 960, Event 20: NC pi0. E_nu=0.998200, vtx_nu=(141.997116,-91.625053,45.375076)
Run 71655